INSTALL
- FAISS - 
- Sent2Vec

In [35]:
import numpy as np
import faiss   

In [36]:
# import os
# import time
# name = "TED-"+ str(time.strftime("%Y%m%d-%H%M"))

# !mkdir $name
# path = os.getcwd() + "/" + name
# os.chdir(path)

# !pwd


# os.chdir(path)


import os
os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-01-21 13:35:05--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-01-21 13:35:06--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  4,08MB/s    in 2,1s    

2021-01-21 13:35:10 (4,08 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/9044100]

IWSLT15.TED.tst2013.en-vi.en
IWSLT15.

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x 3 lw lw     4096 Thg 1  21 13:35 .
drwxrwxr-x 8 lw lw     4096 Thg 1  21 13:35 ..
drwxrwxr-x 2 lw lw     4096 Thg 1  21 13:35 data_bin
-rw-rw-r-- 1 lw lw  1260181 Thg 1  21 13:35 en_test
-rw-rw-r-- 1 lw lw 10011509 Thg 1  21 13:35 en_train
-rw-rw-r-- 1 lw lw  1249342 Thg 1  21 13:35 en_valid
-rw-rw-r-- 1 lw lw  9044100 Thg 1  21 13:35 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 lw lw  1624771 Thg 1  21 13:35 vi_test
-rw-rw-r-- 1 lw lw 12896000 Thg 1  21 13:35 vi_train
-rw-rw-r-- 1 lw lw  1609835 Thg 1  21 13:35 vi_valid


In [90]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# # model = SentenceTransformer('LaBSE')
# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("This is a red cat with a hat.")
# emb2 = model.encode("Have you seen my red cat?")

# cos_sim = util.pytorch_cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)

In [88]:
# print(cos_sim.numpy()[0][0])

0.56247866


In [93]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

RuntimeError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 1.96 GiB total capacity; 379.91 MiB already allocated; 15.81 MiB free; 512.00 MiB reserved in total by PyTorch)

Test Cosin similarity

In [ ]:
import faiss

start = time.time()
vec_lst_srcfile2 = list(vec_lst_srcfile) 
x = vec_lst_srcfile
q = vec_lst_srcfile2
index = faiss.IndexFlatL2(600)
# index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(x)
faiss.normalize_L2(vec_lst_srcfile2)
k = 6
distance, index = index.search(vec_lst_srcfile2, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

In [ ]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

In [ ]:
start = time.time()
saperate = " || "
threshold = 0.8
src_label = "S"
emb_label = "E"

out_file_write = open("en_train_EM", 'w')
out_file_write_score = open("en_train_EM_score", 'w')
out_file_write_factor = open("en_train_EM_factor", 'w')

for i in range(len(lst_srcfile)):
    score = distance[i][1]
    content = ""
    content_label_arr = []
    if score >= threshold:
        best_simi_index = index[i][1]
        content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()
        
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label.append(emb_label)
        content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())
        
        content_label_arr = " ".join(content_label)
        
        
   
    else:
        content = lst_srcfile[i].strip()
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label_arr = " ".join(content_label)
        
        
    if i<10:
        print(content)
        print(len(content.split()))  
        print(len(content_label))
        print("======================")
        

    out_file_write.writelines(content+ "\n")    
    out_file_write_factor.writelines(content_label_arr + "\n")
    out_file_write_score.writelines(str(score) + "\n")
        

out_file_write.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)